In [50]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [51]:
# Path to translation file
path_to_data = 'C:\\Users\\tessw\\OneDrive\\Documents\\University\\Honours\\hau.txt'

# Read file
translation_file = open(path_to_data, "r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]

In [52]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = string.punctuation + "!" + '?'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [53]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [54]:
# Clean sentences
english_sentences = [clean_sentence(pair[0]) for pair in pairs]
hausa_sentences = [clean_sentence(pair[1]) for pair in pairs]

# Tokenize words
hau_text_tokenized, hau_text_tokenizer = tokenize(hausa_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length hausa sentence: {}'.format(len(max(hau_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))

# Check language length
hausa_vocab = len(hau_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("Hausa vocabulary is of {} unique words".format(hausa_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

Maximum length hausa sentence: 85
Maximum length english sentence: 66
Hausa vocabulary is of 269 unique words
English vocabulary is of 205 unique words


In [55]:
max_hausa_len = int(len(max(hau_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

hau_pad_sentence = pad_sequences(hau_text_tokenized, max_hausa_len, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

# Reshape data
hau_pad_sentence = hau_pad_sentence.reshape(*hau_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [56]:
input_sequence = Input(shape=(max_hausa_len,))
embedding = Embedding(input_dim=hausa_vocab, output_dim=128,)(input_sequence)

In [57]:
input_sequence = Input(shape=(max_hausa_len,))
embedding = Embedding(input_dim=hausa_vocab, output_dim=128,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)

In [58]:
input_sequence = Input(shape=(max_hausa_len,))
embedding = Embedding(input_dim=hausa_vocab, output_dim=128,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(max_english_len)(encoder)

In [59]:
input_sequence = Input(shape=(max_hausa_len,))
embedding = Embedding(input_dim=hausa_vocab, output_dim=128,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(max_english_len)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)

In [60]:
input_sequence = Input(shape=(max_hausa_len,))
embedding = Embedding(input_dim=hausa_vocab, output_dim=128,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(max_english_len)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)

In [61]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 85)]              0         
                                                                 
 embedding_14 (Embedding)    (None, 85, 128)           34432     
                                                                 
 lstm_16 (LSTM)              (None, 64)                49408     
                                                                 
 repeat_vector_8 (RepeatVect  (None, 66, 64)           0         
 or)                                                             
                                                                 
 lstm_17 (LSTM)              (None, 66, 64)            33024     
                                                                 
 time_distributed_2 (TimeDis  (None, 66, 205)          13325     
 tributed)                                                 

In [68]:
model_results = enc_dec_model.fit(hau_pad_sentence, eng_pad_sentence, batch_size=50, epochs=250)

Epoch 1/250
1/1 [==============================] - 0s 88ms/step - loss: 1.9402 - accuracy: 0.5258
Epoch 2/250
1/1 [==============================] - 0s 77ms/step - loss: 2.0032 - accuracy: 0.5303
Epoch 3/250
1/1 [==============================] - 0s 188ms/step - loss: 1.9868 - accuracy: 0.5288
Epoch 4/250
1/1 [==============================] - 0s 94ms/step - loss: 1.9451 - accuracy: 0.5212
Epoch 5/250
1/1 [==============================] - 0s 83ms/step - loss: 1.9361 - accuracy: 0.5288
Epoch 6/250
1/1 [==============================] - 0s 77ms/step - loss: 1.9680 - accuracy: 0.5242
Epoch 7/250
1/1 [==============================] - 0s 192ms/step - loss: 1.9279 - accuracy: 0.5303
Epoch 8/250
1/1 [==============================] - 0s 77ms/step - loss: 1.9231 - accuracy: 0.5288
Epoch 9/250
1/1 [==============================] - 0s 71ms/step - loss: 1.9165 - accuracy: 0.5288
Epoch 10/250
1/1 [==============================] - 0s 186ms/step - loss: 1.8828 - accuracy: 0.5333
Epoch 11/250
1/1

In [62]:
def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 5
def eng_to_hau():
    print()
    print("The hausa sentence is: {}".format(hausa_sentences[index]))
    print()
    print('The predicted sentence is :')
    print(logits_to_sentence(enc_dec_model.predict(eng_pad_sentence[index:index+1])[0], hau_text_tokenizer))
    print()
    print("The english sentence is: {}".format(english_sentences[index]))
    
def hau_to_eng():
    print("The hausa sentence is: {}".format(hausa_sentences[index]))
    print()
    print('The predicted sentence is :')
    print(logits_to_sentence(enc_dec_model.predict(hau_pad_sentence[index:index+1])[0], eng_text_tokenizer))
    print()
    print("The english sentence is: {}".format(english_sentences[index]))

In [69]:
def eng_to_hau_inp():
    print("Enter word or phrase to translate: ")
    user_input = input()
    input_sentence = clean_sentence(user_input)
    input_text_tokenized, input_text_tokenizer = tokenize(input_sentence)
    max_input_len = int(len(max(input_text_tokenized,key=len)))
    maxlen = len(user_input)
    input_pad_sentence = pad_sequences(input_text_tokenized, max_input_len, padding = "post")
    input_pad_sentence = input_pad_sentence.reshape(*input_pad_sentence.shape, 1)
    
    print()
    print("The english sentence is: {}".format(user_input))
    print()
    print('The predicted hausa sentence is :')
    print(logits_to_sentence(enc_dec_model.predict(input_pad_sentence[index:index+1])[0], hau_text_tokenizer))
    
def hau_to_eng_inp():
    print("Enter word or phrase to translate: ")
    user_input = input()
    input_sentence = clean_sentence(user_input)
    input_text_tokenized, input_text_tokenizer = tokenize(input_sentence)
    max_input_len = int(len(max(input_text_tokenized,key=len)))
    maxlen = len(user_input)
    input_pad_sentence = pad_sequences(input_text_tokenized, max_input_len, padding = "post")
    input_pad_sentence = input_pad_sentence.reshape(*input_pad_sentence.shape, 1)
    
    print()
    print("The hausa sentence is: {}".format(user_input))
    print()
    print('The predicted english sentence is :')
    print(logits_to_sentence(enc_dec_model.predict(input_pad_sentence[index:index+1])[0], eng_text_tokenizer))

In [70]:
print("Do you want to translate Eng-Hau (enter E) or Hau-Eng? (enter H)")
choice = input()

if choice == 'E' or choice == 'e':
    #eng_to_hau()
    eng_to_hau_inp()
elif choice == 'H' or choice == 'h':
    #hau_to_eng()
    hau_to_eng_inp()
else:
    print("Invalid choice, please enter E or H.")

Do you want to translate Eng-Hau (enter E) or Hau-Eng? (enter H)
e
Enter word or phrase to translate: 
hey let's go to temple

The english sentence is: hey let's go to temple

The predicted hausa sentence is :
1/1 [==============================] - 0s 41ms/step
bjp takaddama takaddama takaddama takaddama takaddama takaddama takaddama takaddama delhi delhi firayim firayim firayim ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan ministan
